<a href="https://colab.research.google.com/github/marelaa/Labor_earthquake_time_predection_Model/blob/main/Earthquake_time_predection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

API-application interface

In [2]:
!pip install kaggle

In [3]:
#the path of kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Importing the earthquake dataset

In [4]:
#API to fetch dataset from kaggle,you should acceot the rules of the competition
!kaggle  competitions download -c LANL-Earthquake-Prediction

 99% 2.25G/2.27G [00:27<00:00, 159MB/s]
100% 2.27G/2.27G [00:27<00:00, 88.2MB/s]


In [5]:
# extraction thecompressed dataset
from zipfile import ZipFile
dataset = '/content/LANL-Earthquake-Prediction.zip'
with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('the dataset is extracted')




the dataset is extracted


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import gc, torch
gc.collect()
try:
    torch.cuda.empty_cache()
except:
    pass
# delete zip file after extraction
import os
os.remove("/content/LANL-Earthquake-Prediction.zip")




In [25]:
import pandas as pd
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import  mean_absolute_error

dftrain=pd.read_csv('train.csv',nrows=3000)
print(dftrain.head())

#cleaning data
dftrain_clean=dftrain.dropna()
y=dftrain_clean.pop('time_to_failure')
x=dftrain_clean

#standardization
scaler=StandardScaler()
x_scaled = scaler.fit_transform(x)

# train/test split
X_train, X_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.2, random_state=42)

# Model's defenition
model = keras.Sequential([ keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)), keras.layers.Dense(1)])

#Compile sets up how loss will be calculated during fit(), optimizer=’ADAM’: Adaptive Moment Estimation.It remembers past gradients and adapts learning rate
model.compile(optimizer='adam',loss='mse',)

#Model training
model.fit(X_train, y_train,validation_data=(X_test, y_test),epochs=30,batch_size=32)

#Model evaluation
MSE = model.evaluate(X_test, y_test)
MAE = mean_absolute_error(y_test, model.predict(X_test))
mean= np.mean(y_test)
PE= (MAE/mean)*100
print(f"MSE: {MSE}")
print(f"Percentage of error:{PE:.2f}%")



   acoustic_data  time_to_failure
0             12           1.4691
1              6           1.4691
2              8           1.4691
3              5           1.4691
4              8           1.4691


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 1.8884 - val_loss: 0.6867
Epoch 2/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.4659 - val_loss: 0.1075
Epoch 3/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0574 - val_loss: 0.0114
Epoch 4/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0066 - val_loss: 0.0034
Epoch 5/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0024 - val_loss: 0.0024
Epoch 6/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0020 - val_loss: 0.0020
Epoch 7/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0015 - val_loss: 0.0016
Epoch 8/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0012 - val_loss: 0.0012
Epoch 9/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 9.8449e-04 - val_loss: 0.0010
Epoch 10/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 6.7527e-04 - val_loss: 7.8437e-04
Epoch 11/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 5.8727e-04 - val_loss: 6.3402e-04
Epoch 12/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - l

In [20]:
print(X_train.shape)

(2400, 1)


In [15]:
print(X_test.shape)

(600, 1)
